In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# Lightly adapted from https://github.com/facebookresearch/segment-anything/blob/main/notebooks/automatic_mask_generator_example.ipynb

# Automatically generating object masks with SAM

Since SAM 2 can efficiently process prompts, masks for the entire image can be generated by sampling a large number of prompts over an image.

The class `SAM2AutomaticMaskGenerator` implements this capability. It works by sampling single-point input prompts in a grid over the image, from each of which SAM can predict multiple masks. Then, masks are filtered for quality and deduplicated using non-maximal suppression. Additional options allow for further improvement of mask quality and quantity, such as running prediction on multiple crops of the image or postprocessing masks to remove small disconnected regions and holes.

## Environment Set-up

If running locally using jupyter, first install `SAM 2` in your environment using the installation instructions in the repository.

If running from Google Colab, set `using_colab=True` below and run the cell. In Colab, be sure to select 'GPU' under 'Edit'->'Notebook Settings'->'Hardware accelerator'. Note that it's recommended to use **A100 or L4 GPUs when running in Colab** (T4 GPUs might also work, but could be slow and might run out of memory in some cases).

In [2]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
import pickle

In [ ]:
# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

In [ ]:
np.random.seed(3)

def show_anns(anns, borders=True):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:, :, 3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.5]])
        img[m] = color_mask 
        if borders:
            import cv2
            contours, _ = cv2.findContours(m.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
            # Try to smooth contours
            contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
            cv2.drawContours(img, contours, -1, (0, 0, 1, 0.4), thickness=1) 

    ax.imshow(img)

To run automatic mask generation, provide a version of SAM 2 to the SAM2AutomaticMaskGenerator class. Set the path below to the SAM 2 checkpoint.

In [ ]:
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

sam2_checkpoint = "./checkpoints/sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

sam2 = build_sam2(model_cfg, sam2_checkpoint, device=device, apply_postprocessing=False)

mask_generator = SAM2AutomaticMaskGenerator(sam2)

In [ ]:
!ls /workspace/OneActor/OneActor/experiments/young_woman_with_long_wavy_hair/exp_21ffdc16

In [ ]:
dir_name = "/workspace/OneActor/OneActor/experiments/young_woman_with_long_wavy_hair/exp_21ffdc16"

image = Image.open(f'{dir_name}/target.jpg')
image = np.array(image.convert("RGB"))
masks = mask_generator.generate(image)
for i in range(len(masks)):
    if masks[i]['segmentation'].mean() > 0.5:
        masks.append(masks[i])
        masks[-1]['segmentation'] = masks[-1]['segmentation'] == False
f, axarr = plt.subplots(1,len(masks), figsize=(20, 10))
for n, mask in enumerate(masks):
    axarr[n].imshow(mask['segmentation'])
    axarr[n].set_title(str(n))
plt.show()
n_mask = int(input())
mask = Image.fromarray(masks[n_mask]['segmentation'])
    
mask.save(f'{dir_name}/target_mask.jpg')
plt.figure(figsize=(3, 3))
plt.imshow(image)
plt.imshow(mask, alpha=0.5)

In [ ]:
for _, _, files in os.walk(f'{dir_name}/base'):
    break
for n in range(len(files)):
    if f'base{n}.jpg' in files:
        image = Image.open(f'{dir_name}/base/base{n}.jpg')
        image = np.array(image.convert("RGB"))
        masks = mask_generator.generate(image)
        for i in range(len(masks)):
            if masks[i]['segmentation'].mean() > 0.5:
                masks.append(masks[i])
                masks[-1]['segmentation'] = (masks[-1]['segmentation'] == False)
        f, axarr = plt.subplots(1,len(masks), figsize=(20, 10))
        for m, mask in enumerate(masks):
            axarr[m].imshow(mask['segmentation'])
            axarr[m].set_title(str(m))
        plt.show()
        n_mask = int(input())
        mask = Image.fromarray(masks[n_mask]['segmentation'])
        mask.save(f"{dir_name}/base/base{n}_mask.png")
        plt.figure(figsize=(2, 2))
        plt.imshow(image)
        plt.imshow(mask, alpha=0.5)

In [3]:
torch.cuda.empty_cache()